# Advanced Features Guide

Deep dive into **CursorHoldability**, **Network Groups**, and other advanced pyhdb-rs features.

## Topics Covered

1. Cursor Holdability - Control cursor behavior across transactions
2. Network Groups - HA and Scale-Out routing
3. Transaction Management - Advanced patterns
4. Connection Validation - Health checks

In [ ]:
import polars as pl
from pyhdb_rs import ConnectionBuilder, CursorHoldability, TlsConfig

## Cursor Holdability

Controls whether cursors remain open after `commit()` or `rollback()`.

### Variants

- `CursorHoldability.None_` - Closed on commit AND rollback (default)
- `CursorHoldability.Commit` - Held across commits, closed on rollback
- `CursorHoldability.Rollback` - Held across rollbacks, closed on commit
- `CursorHoldability.CommitAndRollback` - Held across both

### Example: CommitAndRollback

In [ ]:
conn = (ConnectionBuilder()
    .host("hana.example.com")
    .credentials("SYSTEM", "password")
    .cursor_holdability(CursorHoldability.CommitAndRollback)
    .build())

conn.set_autocommit(False)

try:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM SYS.TABLES LIMIT 100")
        
        # Fetch first batch
        rows = cur.fetchmany(50)
        print(f"Batch 1: {len(rows)} rows")
        
        # Commit - cursor stays open
        conn.commit()
        
        # Continue fetching
        more_rows = cur.fetchmany(50)
        print(f"Batch 2: {len(more_rows)} rows")
        
        # Rollback - cursor STILL stays open
        conn.rollback()
        
        # Can still fetch (but result set may be empty)
        final = cur.fetchmany(10)
        print(f"Batch 3: {len(final)} rows")
finally:
    conn.close()

### Use Case: Large Result Set with Intermediate Commits

In [ ]:
def process_large_table():
    conn = (ConnectionBuilder()
        .host("hana.example.com")
        .credentials("SYSTEM", "password")
        .cursor_holdability(CursorHoldability.CommitAndRollback)
        .build())
    
    conn.set_autocommit(False)
    batch_size = 1000
    commit_interval = 5000
    
    try:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM very_large_table")
            
            total = 0
            while True:
                rows = cur.fetchmany(batch_size)
                if not rows:
                    break
                
                # Process batch
                process_batch(rows)
                total += len(rows)
                
                # Commit every 5000 rows to free locks
                if total % commit_interval == 0:
                    conn.commit()
                    print(f"Processed {total} rows")
            
            conn.commit()
            print(f"Total: {total} rows")
    finally:
        conn.close()

def process_batch(rows):
    # Your processing logic
    pass

## Network Groups

Essential for HANA HA and Scale-Out deployments.

### High Availability Cluster

In [ ]:
# Connect to primary network group
conn_primary = (ConnectionBuilder()
    .host("hana-ha-cluster.example.com")
    .credentials("SYSTEM", "password")
    .network_group("ha-primary")
    .tls(TlsConfig.with_system_roots())
    .build())

try:
    reader = conn_primary.execute_arrow(
        """SELECT HOST, PORT, ACTIVE_STATUS 
           FROM SYS.M_LANDSCAPE_HOST_CONFIGURATION"""
    )
    df = pl.from_arrow(reader)
    print("HA Cluster Nodes:")
    print(df)
finally:
    conn_primary.close()

### Scale-Out System

In [ ]:
conn = (ConnectionBuilder()
    .host("hana-scaleout.example.com")
    .credentials("SYSTEM", "password")
    .network_group("data-network")
    .build())

try:
    reader = conn.execute_arrow(
        """SELECT HOST, COORDINATOR_TYPE, INDEXSERVER_ACTUAL_ROLE
           FROM SYS.M_LANDSCAPE_HOST_CONFIGURATION
           ORDER BY HOST"""
    )
    df = pl.from_arrow(reader)
    print("Scale-Out Topology:")
    print(df)
finally:
    conn.close()

### Multi-Network Routing

In [ ]:
# Writes to internal network
conn_write = (ConnectionBuilder()
    .host("hana-ha.example.com")
    .credentials("WRITER", "password")
    .network_group("internal")
    .build())

# Reads from external network
conn_read = (ConnectionBuilder()
    .host("hana-ha.example.com")
    .credentials("READER", "password")
    .network_group("external")
    .build())

try:
    # Write
    with conn_write.cursor() as cur:
        cur.execute("INSERT INTO logs VALUES (?, ?)", [1, "message"])
        conn_write.commit()
    
    # Read
    with conn_read.cursor() as cur:
        cur.execute("SELECT COUNT(*) FROM logs")
        count = cur.fetchone()[0]
        print(f"Total logs: {count}")
finally:
    conn_write.close()
    conn_read.close()

## Connection Validation

In [ ]:
conn = (ConnectionBuilder()
    .host("hana.example.com")
    .credentials("SYSTEM", "password")
    .build())

try:
    # Check with network round-trip
    if conn.is_valid(check_connection=True):
        print("✓ Connection is valid")
    else:
        print("✗ Connection is invalid")
    
    # Quick check (no network)
    if conn.is_valid(check_connection=False):
        print("✓ Internal state valid")
finally:
    conn.close()

## Complete Production Example

In [ ]:
import os

# Production-ready connection with all features
conn = (ConnectionBuilder()
    .host(os.environ["HANA_HOST"])
    .port(int(os.environ.get("HANA_PORT", "30015")))
    .credentials(os.environ["HANA_USER"], os.environ["HANA_PASSWORD"])
    .database(os.environ.get("HANA_DATABASE", "SYSTEMDB"))
    .tls(TlsConfig.from_directory("/etc/hana/certs"))
    .network_group("production")
    .cursor_holdability(CursorHoldability.CommitAndRollback)
    .autocommit(False)
    .build())

try:
    # Validate connection
    if not conn.is_valid():
        raise RuntimeError("Connection validation failed")
    
    # Execute with optimal configuration
    reader = conn.execute_arrow(
        """SELECT * FROM sales_data 
           WHERE fiscal_year = 2026 
           ORDER BY sale_date DESC"""
    )
    df = pl.from_arrow(reader)
    
    print(f"Loaded {len(df):,} records")
    print(df.head())
finally:
    conn.close()

## Best Practices Summary

1. **Cursor Holdability**
   - Use `CommitAndRollback` for large result sets with intermediate commits
   - Default (`None_`) is safest for most use cases

2. **Network Groups**
   - Always configure for HA/Scale-Out deployments
   - Use descriptive names: `production`, `internal`, `backup`

3. **Connection Validation**
   - Validate before critical operations
   - Use `check_connection=False` for quick checks

4. **TLS**
   - Always use in production
   - Prefer `from_directory()` for flexibility